In [1]:
import torch
import torch.nn as nn
import numpy as np

In [2]:
class Ososa(nn.Module):
    def __init__(self):
        super(Ososa, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, (10, 10))
        self.conv2 = nn.Conv2d(6, 4, (8, 8))
        self.conv3 = nn.Conv2d(4, 2, (6, 6))
        self.relu = nn.ReLU()
        self.pool1 = nn.MaxPool2d((3, 3))
        self.pool2 = nn.MaxPool2d((3, 3))
        self.pool3 = nn.MaxPool2d((5, 5))
        self.linear1 = nn.Linear(2 * 5 * 5, 14)
        self.linear2 = nn.Linear(10, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.pool3(x)
        x = torch.flatten(x, 1)
        x = self.linear1(x)
        # x = self.relu(x)
        # x = self.linear2(x)
        # x = self.relu(x)
        return x

In [3]:

def train(x, y, device, batch_count=20, epoch=200, learning_rate=0.005, save_path='./ososa_model.pth'):
    model = Ososa().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    # loss_f = loss_function
    # loss_fn = nn.CrossEntropyLoss()
    # y = y.long()
    # loss_f = loss_function.OsosaLoss(y, number_of_centroids=5)
    loss_f = nn.CrossEntropyLoss()
    # TODO batch halinde alabiliriz daha cok data kullandigimizda
    batch_size = int(x.shape[0] / batch_count)
    for e in range(epoch):
        for b in range(batch_count):
            batched_x = x[b*batch_size:(b+1)*batch_size, :, :, :]
            batched_y = y[b*batch_size:(b+1)*batch_size]
            train_data = torch.from_numpy(batched_x).float().to(device)
            train_labels = torch.from_numpy(batched_y).long().to(device)
            optimizer.zero_grad()
            prediction = model(train_data)
            loss = loss_f(prediction, train_labels)
            loss.backward()
            optimizer.step()

        if e % 10 == 0:
            print("Loss in e={} is {}".format(e, loss))
            #  loss_f.show(prediction, y)
    print('finished')
    
    torch.save(model.state_dict(), save_path)


In [29]:
def test(x, y, device, batch_count=4, load_path='./ososa_model.pth'):
    correct = 0
    total = 0
    
    batch_size = int(x.shape[0] / batch_count)
    for b in range(batch_count):
        batched_x = x[b*batch_size:(b+1)*batch_size, :, :, :]
        batched_y = y[b*batch_size:(b+1)*batch_size]

        test_data = torch.from_numpy(batched_x).float().to(device)
        test_labels = torch.from_numpy(batched_y).long().to(device)
        with torch.no_grad():
            model = Ososa().to(device)
            model.load_state_dict(torch.load(load_path))
            outputs = model(test_data)
            _, predicted = torch.max(outputs.data, 1)
            for p, l in zip(predicted, test_labels):
                total += 1
                if p.data == l.data:
                    correct += 1

    print("Total:", total)
    print("Correct:", correct)
    print("Accuracy:", correct/total)

In [5]:
def take_data(input_directory_1="./numpy_images", input_directory_2="./labels", limit=5000):
	directory_list = os.listdir(input_directory_1)
	data = []
	labels = []
	for i, image_name in enumerate(directory_list):
		try:
			datum = np.load("{}/{}".format(input_directory_1, image_name))
			datum = datum.reshape(3, 300, 300)
			label = np.load("{}/{}".format(input_directory_2, image_name))
			data.append(datum)
			labels.append(label)
		except ValueError:
			# print('valueError')
			pass
		if i % 1000 == 0:
			print(i)
		if i == limit:
			break
	return np.array(data), np.array(labels)

In [6]:
data, labels = take_data()

0
1000
2000
3000
4000
5000


In [7]:
data_limit = 4000
train_data = data[:data_limit, :, :, :]
test_data = data[data_limit:, :, :, :]
train_labels = labels[:data_limit]
test_labels = labels[data_limit:]

In [8]:
if torch.cuda.is_available:
    device = 'cuda'
else:
    device = 'cpu'


In [9]:

train(train_data, train_labels, device, batch_count=10, epoch=50)


Loss in e=0 is 2.633272409439087
Loss in e=10 is 2.4129557609558105
Loss in e=20 is 2.3440566062927246
Loss in e=30 is 2.324758529663086
Loss in e=40 is 2.3185789585113525
finished


In [30]:
test(test_data, test_labels, device)
    

Total: 956
Correct: 150
Accuracy: 0.15690376569037656
